In [1]:
"""
DJOSHI
Devavrat Joshi
"""

'\nDJOSHI\nDevavrat Joshi\n'

In [2]:
'''
    Block for definining common Program Parameters & Constants Definitions
'''

CLICKBAITS_PATH = ""
REMOVE_PUNCTUATIONS = False
REMOVE_DEMOJI = True
TOP_N_ROUGE_SCORES = 8

PERFORM_SENTENCE_EXTRACTION = True
PERFORM_DOCUMENT_QA = True
PERFORM_SENTENCE_QA = True

USE_BEST_CONFIGURED_SETTINGS = True


In [3]:
"""
Sentence Extraction (SE) Constant Definitions Block
"""

SE_BERT_SEQSEQ_LABEL_CLASSIFCATION = False
SE_MODELS = []

if SE_BERT_SEQSEQ_LABEL_CLASSIFCATION == True:
  SE_MODELS.append({"name":"bert-base-uncased", "features_dim": 384, "pooling": "mean"})
else:
  SE_MODELS.append({"name":"sentence-transformers/paraphrase-MiniLM-L3-v2", "features_dim": 384, "pooling": "mean"})
  SE_HIDDEN_DIM = 1024

# 0 means dont use, 1 means calculate based on training data and [1,4] or anything else use provided classweights array  
SE_USE_CLASSWEIGHTS = 1

SE_BALANCE_NEG_MULTIPLIER = 0
SE_OPTIMIZER_NAME = "AdamW"
SE_LOSS_FUNCTION_NAME = "BCELoss"

SE_BERT_NUM_HIDDEN_LAYERS = 3
SE_BERT_NUM_ATTENTION_HEADS = 12
SE_DROPOUT_HIDDEN = 0.1
SE_DROPOUT_ATTENTION = 0.1
SE_MAX_LEN = 512
SE_BATCH_SIZE = 8
SE_EPOCH_MAX = 20
SE_DROPOUT =0.5
SE_LEARNING_RATE = 5e-5
SE_WEIGHT_DECAY  = 5e-05
SE_THRESHOLD = 0.5

In [4]:
'''
    Question Answering (QA) Block for definining Program Parameters & Constants Definitions
'''

#Question Answering (QA) Parameters
QA_MODELS = []
#QA_MODELS.append({'name':'bert-base-uncased'})
QA_MODELS.append({'name':'roberta-base'})
#QA_MODELS.append({'name':'bert-large-uncased'})


QA_TRAIN_BATCH_SIZE = 16
QA_VAL_BATCH_SIZE = 8

QA_BERT_NUM_HIDDEN_LAYERS = 12
QA_BERT_NUM_ATTENTION_HEADS = 12
QA_OPTIMIZER_NAME = "AdamW"
QA_LEARNING_RATE_SCHEDULER = False
QA_DROPOUT_HIDDEN = 0.4
QA_DROPOUT_ATTENTION = 0.4
QA_LEARNING_RATE = 3e-5
QA_WEIGHT_DECAY = 0.01
QA_EPOCH_MAX=30



In [5]:
'''
Override above settings for best configuration
'''
def set_se_configuration():
  if USE_BEST_CONFIGURED_SETTINGS == True:
      global TOP_N_ROUGE_SCORES
      global SE_BERT_SEQSEQ_LABEL_CLASSIFCATION
      global SE_DROPOUT_HIDDEN
      global SE_DROPOUT_ATTENTION
      global SE_DROPOUT
      global SE_LEARNING_RATE
      global SE_WEIGHT_DECAY
      global SE_EPOCH_MAX
      global SE_USE_CLASSWEIGHTS

      SE_USE_CLASSWEIGHTS = 0
      TOP_N_ROUGE_SCORES = 8
      SE_BERT_SEQSEQ_LABEL_CLASSIFCATION = False
      SE_DROPOUT_HIDDEN = 0.1
      SE_DROPOUT_ATTENTION = 0.1
      SE_DROPOUT =0.5
      SE_LEARNING_RATE = 5e-5
      SE_WEIGHT_DECAY  = 5e-05
      SE_EPOCH_MAX = 10  
        
def set_qa_document_configuration():
  if USE_BEST_CONFIGURED_SETTINGS == True:
      global TOP_N_ROUGE_SCORES
      global QA_DROPOUT_HIDDEN
      global QA_DROPOUT_ATTENTION
      global QA_LEARNING_RATE
      global QA_WEIGHT_DECAY
      global QA_EPOCH_MAX
      global QA_MODELS

      TOP_N_ROUGE_SCORES = 0
      QA_DROPOUT_HIDDEN = 0.2
      QA_DROPOUT_ATTENTION = 0.2
      QA_LEARNING_RATE = 3e-5
      QA_WEIGHT_DECAY = 0.01
      QA_EPOCH_MAX=10
      QA_MODELS = []
      QA_MODELS.append({'name':'roberta-base'})

        
def set_qa_statement_configuration():
  if USE_BEST_CONFIGURED_SETTINGS == True:
      global TOP_N_ROUGE_SCORES
      global QA_DROPOUT_HIDDEN
      global QA_DROPOUT_ATTENTION
      global QA_LEARNING_RATE
      global QA_WEIGHT_DECAY
      global QA_EPOCH_MAX
      global QA_MODELS
      
      TOP_N_ROUGE_SCORES = 0
      QA_DROPOUT_HIDDEN = 0.3
      QA_DROPOUT_ATTENTION = 0.3
      QA_LEARNING_RATE = 3e-5
      QA_WEIGHT_DECAY = 0.01
      QA_EPOCH_MAX=12
      QA_MODELS = []
      QA_MODELS.append({'name':'roberta-base'})

In [6]:
'''
    Block for loading training data and creating dataset and data loaders
'''
import pandas as pd
import os
import torch
import numpy as np
from string import punctuation

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


try:
  from unidecode import unidecode
except ModuleNotFoundError:
  !pip install unidecode
  from unidecode import unidecode
  
try:
  import operator
except ModuleNotFoundError:
  !pip install operator

try:
  from rouge import Rouge
except ModuleNotFoundError:
  !pip install rouge
  from rouge import Rouge

!pip install demoji
import demoji

demoji.download_codes()
rouge_score = Rouge()

def getby_uuid(df, uuid):
  return df.loc[df['uuid'] == uuid]

""" In some cases the spoilerPositions has defined targetParagraph index as -1.
  I have observed in such cases the spoiler text is extracted from targetTitle.
  Hence, i am appending targetTitle to targetParagraphs. -1 index will automatically take the last appended element from the tgp list
"""
def append_targetTitle_to_targetParagraphs(record):
  targetParagraphs = record['targetParagraphs']
  
  append_title = False
  for spoilerPosition in record['spoilerPositions']:
      tgpStartIndex = spoilerPosition[0][0]
      if tgpStartIndex == -1:
          append_title = True
          break
      
      if len(spoilerPosition) == 2 :
          tgpEndIndex = spoilerPosition[1][0]
          if tgpEndIndex == -1:
              append_title = True
              break

  if append_title == True:
      targetParagraphs.append(record['targetTitle'])
  return targetParagraphs

"""
  targetParagraphs is used as single source for creating document and spoiler. Hence targetParagraphs needs to be curated. 
  Curation involves
    1. replacing special character quotes with regualar quotes
    2. trim trailing and leading whitespaces
    3. convert to lower case
"""
def curate_text_array(texts):
  textCurated = []
  for text in texts:
    text = curate_text(text)
    textCurated.append(text)
  
  return textCurated

def curate_text(text):
    text = demoji_text(text)
    text = unidecode(text)
    text = text.replace('’','\'') # data contains special character quotes which messes up tokenization
    if REMOVE_PUNCTUATIONS == True:
        punct_removed_tokens = []
        tokens = text.split()
        for token in tokens:
            token = token.strip()
            token = token.strip(punctuation)
            if len(token) > 0:
                punct_removed_tokens.append(token)
        text = " ".join(punct_removed_tokens)
        
    text = text.strip().lower()
    return text

def demoji_text(text):
  if REMOVE_DEMOJI == True:
    dem = demoji.findall(text)
    for item in dem.keys():
        text = text.replace(item, '')
  return text

"""
Create document by concatenating targetParagraphs (tgp)
This document and each targetParagraph is fed as an input pair to the sentence extraction model
"""
def create_tgp_document(record):
    topn_targetParagraphs = []
    
    if TOP_N_ROUGE_SCORES > 0:
        scores = record['tgpRougeScore']
        topn_max  = len(scores) if len(scores) < TOP_N_ROUGE_SCORES else TOP_N_ROUGE_SCORES
        topn_scores = scores[0:topn_max]
        topn_indexes = [score['index'] for score in topn_scores]
        topn_indexes.sort()
    
        
        for topn_index in topn_indexes:
            tgp = record['targetParagraphs'][topn_index]
            topn_targetParagraphs.append(tgp)
    else:
        topn_targetParagraphs = record['targetParagraphs']
    
    return " ".join(topn_targetParagraphs)

def string_list_to_string(str_lst):
    return str_lst[0]

# Mark each targetParagraph with 0 (Not to be extracted) and 1 (to be extracted) label depending on definition in spoilerPosition
def mark_tgp_extraction_label(record):
    tgpSELabels = [0] * record['tgpsCount']
    
    for spoilerPosition in record['spoilerPositions']:
        tgpSELabels[spoilerPosition[0][0]] = 1
        if len(spoilerPosition) == 2:
            tgpSELabels[spoilerPosition[1][0]] = 1
    return tgpSELabels

def calc_rouge_score(record):
    scores = [{}] * record['tgpsCount']
    for i in range(record['tgpsCount']):
        try:
            score = rouge_score.get_scores(record['postText'], record['targetParagraphs'][i])
            scores[i] = {"index":i, "score": score[0]['rouge-l']['p']}
        except Exception as ex:
            scores[i] = {"index":i, "score": 0}

    scores.sort(key=operator.itemgetter('score'), reverse = True)
    
    return scores

def assing_tgpId(record):
    uuid = record['uuid']
    tgpIds = []
    for i in range(len(record['targetParagraphs'])):
      tgpId = (uuid, i)
      tgpIds.append(tgpId)
    return tgpIds

def append_from_source(record, val_df):
  tgpId = record['tgpId']
  uuid = tgpId[0]
  statementId = tgpId[1]
  val_df_record = getby_uuid(val_df, uuid)
  val_df_record.reset_index(drop=True,inplace=True)
  postTexts = val_df_record['postText'][0]
  targetParagraphs = val_df_record['targetParagraphs'][0][statementId]
  tgpDocuments = val_df_record['tgpDocument'][0]
  tgpQALabels = val_df_record['tgpQALabel'][0][statementId]
  spoiler = val_df_record['spoiler'][0]
  spoilerPositions = val_df_record['spoilerPositions'][0]
  tgpsCount = val_df_record['tgpsCount'][0]
  return pd.Series([postTexts, targetParagraphs, tgpDocuments, spoiler, spoilerPositions, tgpsCount, tgpQALabels])

def append_info_to_se_output(val_df, se_outputs_df):
  
  se_outputs_df[['postText','targetParagraphs','tgpDocument','spoiler','spoilerPositions','tgpsCount', 'tgpQALabel']] = se_outputs_df.apply(append_from_source, val_df=val_df, axis=1)
  se_outputs_df.reset_index(level=0, inplace=True)
  se_outputs_df.to_csv('output_se.csv')

def load_data(filename):
    df = pd.read_json(os.path.join(CLICKBAITS_PATH, filename), lines=True)
    
    df['tags'] = df['tags'].apply(string_list_to_string)
    df['targetParagraphs'] = df.apply(append_targetTitle_to_targetParagraphs, axis=1)
    df['targetParagraphs'] = df['targetParagraphs'].apply(curate_text_array)
    df['tgpId'] = df.apply(assing_tgpId, axis = 1)
    df['postText'] = df['postText'].apply(curate_text_array)
    df['postText'] = df['postText'].apply(string_list_to_string)
    df['spoiler'] = df['spoiler'].apply(curate_text_array)
    df['tgpsCount'] = df['targetParagraphs'].apply(len)
    
    df['tgpSELabel'] = df.apply(mark_tgp_extraction_label, axis=1)
    df['tgpRougeScore'] = df.apply(calc_rouge_score, axis =1)

    df['tgpDocument'] = df.apply(create_tgp_document, axis = 1)

    df.reset_index(level=0, inplace=True)
    phrase_passage_df = df[df.tags != 'multi']
    multi_df = df[df.tags == 'multi']

    return df, phrase_passage_df, multi_df
    

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 7.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.5 MB/s 


In [7]:
'''
  Sentence Extraction 
    Sentence Extraction Block for  :
        1.  Dataset and Dataloader cell 
        2.  SentenceBertClass Model Definition
        2.  factory method of model creation
        3.  train loop - train_epoch and validate_epoch
'''
from torch.utils.data import Dataset, DataLoader 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook
import math
import numpy as np
from sklearn.utils import class_weight

try:
  from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
except ModuleNotFoundError:
  !pip install transformers
  from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

# Dataset for ClickBaitsWebis22 (CBW22) Sentence Extraction (SE)
class CBW22SEDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        sentence = str(self.data.iloc[index].targetParagraphs)
        document = str(self.data.iloc[index].postText)
        tgpId = self.data.iloc[index].tgpId

        inputs = self.tokenizer.batch_encode_plus(
            [sentence, document], 
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'tgpId': tgpId,
            'sent_ids': torch.tensor(ids[0], dtype=torch.long),
            'doc_ids': torch.tensor(ids[1], dtype=torch.long),
            'sent_mask': torch.tensor(mask[0], dtype=torch.long),
            'doc_mask': torch.tensor(mask[1], dtype=torch.long),
            'targets': torch.tensor([self.data.iloc[index].tgpSELabel], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

def balance_data(df):

  print("Pre-balancing dataset size: {}".format(len(df)))

  pos_df = df[df.tgpSELabel == 1]
  neg_df = df[df.tgpSELabel == 0]

  print("Negative sample size:", len(neg_df))
  print("Positive sample size:", len(pos_df))

  sub_neg_df = neg_df.sample(len(pos_df)*SE_BALANCE_NEG_MULTIPLIER) 
  balanced_df = pos_df.append(sub_neg_df)

  print("Balanced dataset size: {}".format(len(balanced_df)))
  
  return balanced_df

def get_data_se(tokenizer, df, balance = False, cal_class_weights = False):
  classWeightsTensor = None

  if balance == True:
    df = balance_data(df)
  
  df['tgpQALabel'] = df.apply(compute_qa_answer_label_statement, axis = 1)
  df['topNRougeStatement'] = df.apply(mark_rouge_topn_tgp_for_sentence_qa, axis = 1)
  
  col_list = ['tgpId','postText', 'targetParagraphs', 'tgpDocument', 'tgpSELabel', 'tgpQALabel', 'topNRougeStatement']
  df = df[col_list]
  
  df = df.apply(pd.Series.explode)
  # get only those targetParagraphs which appear in spoiler
  df = df[df.topNRougeStatement == True]

  if cal_class_weights == True:
    classWeightsTensor = getClassWeights(df)

  ds = CBW22SEDataset(df, tokenizer, SE_MAX_LEN)
  dl = DataLoader(batch_size=SE_BATCH_SIZE, shuffle=True, dataset=ds)
  return dl, classWeightsTensor


# calculate weights of lables as their imbalance in distribution
def getClassWeights(df):
  classWeightsTensor = None
  if SE_USE_CLASSWEIGHTS != 0:
    if SE_USE_CLASSWEIGHTS == 1:
        Y = df['tgpSELabel'].explode()
        uniqueY = df.tgpSELabel.explode().unique()
        classWeights = class_weight.compute_class_weight(class_weight="balanced", classes=uniqueY, y=Y)
    else:
        classWeights = SE_USE_CLASSWEIGHTS
    print("Using class Weights: {}".format(classWeights))    
    classWeightsTensor = torch.tensor(classWeights, dtype=torch.float32).to(device)
  return classWeightsTensor

# get mean pooling for sentence bert models 
# ref https://www.sbert.net/examples/applications/computing-embeddings/README.html#sentence-embeddings-with-transformers
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def max_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    token_embeddings[input_mask_expanded == 0] = -1e9  # Set padding tokens to large negative value
    return torch.max(token_embeddings, 1)[0]

def cls_pooling(model_output, attention_mask):
    return model_output[0][:,0]

'''
  customized model with a drop out and a dense layer on top of distil bert to get the final output for the model. 
'''
class SentenceBertClass(torch.nn.Module):
    def __init__(self, configuration, sentence_model_name, features_dim, pooling):
        super(SentenceBertClass, self).__init__()
        self.pooling = pooling
        self.l1 = AutoModel.from_pretrained(pretrained_model_name_or_path = sentence_model_name, config = configuration)
        self.pre_classifier = torch.nn.Linear(features_dim * 3, SE_HIDDEN_DIM)
        self.dropout = torch.nn.Dropout(SE_DROPOUT)
        self.classifier = torch.nn.Linear(SE_HIDDEN_DIM, 1)
        self.classifierSigmoid = torch.nn.Sigmoid()

    def forward(self, sent_ids, doc_ids, sent_mask, doc_mask):
        sent_output = self.l1(input_ids=sent_ids, attention_mask=sent_mask) 

        sentence_embeddings = sent_output
        if self.pooling == "mean":
          sentence_embeddings = mean_pooling(sent_output, sent_mask) 
        elif self.pooling == "cls":
          sentence_embeddings = cls_pooling(sent_output, sent_mask)
        elif self.pooling == "max":
          sentence_embeddings = max_pooling(sent_output, sent_mask)

        doc_output = self.l1(input_ids=doc_ids, attention_mask=doc_mask) 

        doc_embeddings = doc_output
        if self.pooling == "mean":
          doc_embeddings = mean_pooling(doc_output, doc_mask)
        elif self.pooling == "cls":
          doc_embeddings = cls_pooling(doc_output, doc_mask)
        elif self.pooling == "max":
          doc_embeddings = max_pooling(doc_output, doc_mask)

        # elementwise product of sentence embs and doc embs
        combined_features = sentence_embeddings * doc_embeddings

        # Concatenate input features and their elementwise product
        concat_features = torch.cat((sentence_embeddings, doc_embeddings, combined_features), dim=1)   

        pooler = self.pre_classifier(concat_features) 
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = self.classifierSigmoid(output) 

        return output


def create_model_se(sentence_model_name, features_dim, pooling):
    
    tokenizer = AutoTokenizer.from_pretrained(sentence_model_name) 
    
    configuration = AutoConfig.from_pretrained(sentence_model_name)
    configuration.hidden_dropout_prob = SE_DROPOUT_HIDDEN
    configuration.attention_probs_dropout_prob = SE_DROPOUT_ATTENTION
    configuration.num_hidden_layers = SE_BERT_NUM_HIDDEN_LAYERS
    configuration.num_attention_heads = SE_BERT_NUM_ATTENTION_HEADS

    if SE_BERT_SEQSEQ_LABEL_CLASSIFCATION == True:
      model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path = sentence_model_name, config = configuration)
    else:
      model = SentenceBertClass(configuration, sentence_model_name, features_dim, pooling)

    model.to(device)
    
    if SE_LOSS_FUNCTION_NAME == "BCELoss":
        if SE_USE_CLASSWEIGHTS == 0:
          loss_function = nn.BCELoss().to(device)
        else:
          loss_function = nn.BCELoss(reduction='none').to(device)
  
    if SE_OPTIMIZER_NAME == "SGD":
      optimizer = optim.SGD(model.parameters(), lr=SE_LEARNING_RATE, weight_decay=SE_WEIGHT_DECAY)
    elif SE_OPTIMIZER_NAME == "Adam":
      optimizer = optim.Adam(model.parameters(), lr=SE_LEARNING_RATE, weight_decay=SE_WEIGHT_DECAY)
    elif SE_OPTIMIZER_NAME == "AdamW":
      optimizer = optim.AdamW(model.parameters(), lr=SE_LEARNING_RATE, weight_decay=SE_WEIGHT_DECAY)
    
    return model, loss_function, optimizer, tokenizer

def train_epoch_se(epoch, model, loss_function, optimizer, train_loader, classWeightsTensor):    
    tr_loss = []
    f1scores = []
    
    model.train()
    pbar = tqdm_notebook(train_loader, desc="SE Training batch for epoch-{}".format(epoch))
    for _,data in enumerate(pbar):
        sent_ids = data['sent_ids'].to(device, dtype = torch.long)
        doc_ids = data['doc_ids'].to(device, dtype = torch.long)
        sent_mask = data['sent_mask'].to(device, dtype = torch.long)
        doc_mask = data['doc_mask'].to(device, dtype = torch.long) 
        targets = data['targets'].to(device, dtype = torch.float)  

        outputs = model(sent_ids, doc_ids, sent_mask, doc_mask) 
        loss = loss_function(outputs, targets)
        
        if SE_USE_CLASSWEIGHTS != 0:
            batch_class_weight = classWeightsTensor[targets.data.view(-1).long()].view_as(targets)
            loss_class_weighted = loss * batch_class_weight
            loss = loss_class_weighted.mean()
            batch_loss = loss.item()
        else:
            batch_loss = loss.item()

        tr_loss.append(batch_loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        targets = targets.cpu().numpy()
        outputs = outputs.detach().cpu().numpy()
        outputs_pred = [[0.]] * len(outputs)
        for i in range(len(outputs)):
            if outputs[i][0] > SE_THRESHOLD:
                outputs_pred[i][0] = 1.
                
        batch_f1score = f1_score(targets, outputs_pred, average='micro') * 100
        f1scores.append(batch_f1score)

        pbar.set_postfix_str("Loss: {}, F1 Score:{}".format(round(batch_loss,2), round(batch_f1score,2)))
        

    epoch_loss = 0
    epoch_f1score = 0
    if len(tr_loss) > 0 :
      epoch_loss = sum(tr_loss)/len(tr_loss)
    
    if len(f1scores) > 0:
      epoch_f1score = sum(f1scores)/len(f1scores)
    
    return epoch_loss, epoch_f1score

def validate_epoch_se(epoch, model, loss_function, optimizer, dl, classWeightsTensor):
    losses = []
    f1scores = []
    val_outputs = []

    model.eval()
    pbar = tqdm_notebook(dl, desc="SE Validation batch for epoch-{}".format(epoch))
    with torch.no_grad():
        for _, data in enumerate(pbar): 
            
            tgpIds = data['tgpId']
            sent_ids = data['sent_ids'].to(device, dtype = torch.long)
            doc_ids = data['doc_ids'].to(device, dtype = torch.long)
            sent_mask = data['sent_mask'].to(device, dtype = torch.long)
            doc_mask = data['doc_mask'].to(device, dtype = torch.long) 
            targets = data['targets'].to(device, dtype = torch.float)  

            outputs = model(sent_ids, doc_ids, sent_mask, doc_mask) 
            loss = loss_function(outputs, targets)
            
            if SE_USE_CLASSWEIGHTS != 0:
                batch_class_weight = classWeightsTensor[targets.data.view(-1).long()].view_as(targets)
                loss_class_weighted = loss * batch_class_weight
                loss = loss_class_weighted.mean()
                batch_loss = loss.item()
            else:
                batch_loss = loss.item()
            
            losses.append(batch_loss)
            
            targets = targets.cpu().numpy()
            outputs = outputs.detach().cpu().numpy()
            outputs_pred = [[0.]] * len(outputs)
            for i in range(len(outputs)):
                if outputs[i][0] > SE_THRESHOLD:
                    outputs_pred[i][0] = 1.
            
            batch_f1score = f1_score(targets, outputs_pred, average='micro') * 100
            f1scores.append(batch_f1score)

            pbar.set_postfix_str("Loss: {}, F1 Score:{}".format(round(batch_loss,2), round(batch_f1score,2)))

            for index in range(len(tgpIds[0])):
                val_outputs.append({
                    "tgpId": (tgpIds[0][index],tgpIds[1][index].item()), 
                    "tgpSELabel": int(targets[index][0]), 
                    "sePredLabel": int(outputs_pred[index][0])
                })

    epoch_loss = 0
    epoch_f1score = 0
    if len(losses) > 0 :
      epoch_loss = sum(losses)/len(losses)
    
    if len(f1scores) > 0:
      epoch_f1score = sum(f1scores)/len(f1scores)
    
    return epoch_loss, epoch_f1score, val_outputs

'''
  Sentence Extraction using BERT Sequence Label Classification model
'''

def train_epoch_se_bertlc(epoch, model, loss_function, optimizer, train_loader, classWeightsTensor):    
    tr_loss = []
    nb_tr_examples = 0
    n_correct = 0
    f1scores = []
    
    model.train()
    pbar = tqdm_notebook(train_loader, desc="Training batch for epoch-{}".format(epoch))
    for _,data in enumerate(pbar):
        targets = data['labels']
        
        outputs = model(**data)
        
        loss = outputs.loss

        if SE_USE_CLASSWEIGHTS != 0:
            batch_class_weight = classWeightsTensor[targets.data.view(-1).long()].view_as(targets)
            loss_class_weighted = loss * batch_class_weight
            loss = loss_class_weighted.mean()
            batch_loss = loss.item()
        else:
            batch_loss = loss.item()
            
        tr_loss.append(batch_loss)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        logits = outputs.logits
        outputs_pred = (logits > SE_THRESHOLD)
        batch_correct = torch.count_nonzero(targets == outputs_pred).item()
        n_correct += batch_correct
        nb_tr_examples+=targets.size(0)

        batch_accuracy = (batch_correct * 100)/targets.size(0)
        batch_f1score = f1_score(targets.cpu(), outputs_pred.cpu(), average='micro') * 100
        f1scores.append(batch_f1score)

        pbar.set_postfix_str("Loss: {}, F1 Score:{}, Accuracy:{}".format(round(batch_loss,2), round(batch_f1score,2), round(batch_accuracy,2)))
        

    epoch_loss = sum(tr_loss)/len(tr_loss)
    epoch_f1score = sum(f1scores)/len(f1scores)
    epoch_accuracy = (n_correct*100)/nb_tr_examples
    
    return epoch_loss, epoch_f1score, epoch_accuracy

def validate_se_bertlc(epoch, model, loss_function, optimizer, dl, classWeightsTensor):
    losses = []
    model.eval()
    nb_tr_examples = 0
    n_correct = 0
    f1scores = []

    pbar = tqdm_notebook(dl, desc="Validation batch for epoch-{}".format(epoch))
    with torch.no_grad():
        for _, data in enumerate(pbar): 
            
            targets= data['labels']
            outputs = model(**data)
            
            loss = outputs.loss
            
            if SE_USE_CLASSWEIGHTS != 0:
                batch_class_weight = classWeightsTensor[targets.data.view(-1).long()].view_as(targets)
                loss_class_weighted = loss * batch_class_weight
                loss = loss_class_weighted.mean()
                batch_loss = loss.item()
            else:
                batch_loss = loss.item()
                
            losses.append(batch_loss)
            
            logits = outputs.logits
            outputs_pred = (logits > SE_THRESHOLD)
            batch_correct = torch.count_nonzero(targets == outputs_pred).item()
            n_correct += batch_correct
            nb_tr_examples+=targets.size(0)

            batch_accuracy = (batch_correct * 100)/targets.size(0)

            batch_f1score = f1_score(targets.cpu(), outputs_pred.cpu(), average='micro') * 100
            f1scores.append(batch_f1score)

            pbar.set_postfix_str("Loss: {}, F1 Score:{}, Accuracy:{}".format(round(batch_loss,2), round(batch_f1score,2), round(batch_accuracy,2)))

    epoch_loss = sum(losses)/len(losses)
    epoch_f1score = sum(f1scores)/len(f1scores)
    epoch_accuracy = (n_correct*100)/nb_tr_examples
    
    return epoch_loss, epoch_f1score, epoch_accuracy


def train_se(model, loss_function, optimizer, tokenizer, train_loader, val_loader, val_df, classWeightsTensor):
    train_losses = []
    train_f1scores = []
    val_losses = []
    val_f1scores = []
    epochs = np.arange(SE_EPOCH_MAX)
    val_outputs = []

    pbar = tqdm_notebook(range(SE_EPOCH_MAX), desc="Epoch", position = 0)
    for epoch in pbar:

        if SE_BERT_SEQSEQ_LABEL_CLASSIFCATION == True:
          train_epoch_loss, train_epoch_f1score, epoch_accuracy = train_epoch_se_bertlc(epoch, model, loss_function, optimizer, train_loader, classWeightsTensor)
        else:
          train_epoch_loss, train_epoch_f1score  = train_epoch_se(epoch, model, loss_function, optimizer, train_loader, classWeightsTensor)
        
        train_losses.append(train_epoch_loss)
        train_f1scores.append(train_epoch_f1score)
        
        if SE_BERT_SEQSEQ_LABEL_CLASSIFCATION == True:
          val_epoch_loss, val_epoch_f1score, epoch_accuracy = validate_se_bertlc(epoch, model, loss_function, optimizer, val_loader, classWeightsTensor)
        else:
          val_epoch_loss, val_epoch_f1score, val_outputs = validate_epoch_se(epoch, model, loss_function, optimizer, val_loader, classWeightsTensor)

        val_losses.append(val_epoch_loss)
        val_f1scores.append(val_epoch_f1score)

        pbar.set_postfix_str("Training loss: {} / F1 Score: {}, Validation Loss: {} / F1 Score: {}".format(
                                round(train_epoch_loss,2), round(train_epoch_f1score,2), 
                                round(val_epoch_loss,2), round(val_epoch_f1score,2)))

    scores = {}
    scores['Error'] = 'None'
    scores['Training Loss'] = round(train_losses[-1], 2)
    scores['Training F1Score'] = round(train_f1scores[-1], 2)
    scores['Validation Loss'] = round(val_losses[-1], 2)
    scores['Validation F1Score'] = round(val_f1scores[-1], 2)

    plt.plot(epochs, train_losses, color = 'blue')
    plt.plot(epochs, val_losses, color = 'red')
    plt.axhline(y=scores['Training Loss'], linestyle='dotted', color = 'blue')
    plt.axhline(y=scores['Validation Loss'], linestyle='dotted', color = 'red')
    plt.gca().legend(('Training Loss: {}'.format(scores['Training Loss']),
                      'Validation Loss: {}'.format(scores['Validation Loss'])))
    plt.title("Training-Validation Loss Curve - Sentence Extraction")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.savefig('epoch_trainval_losses_se.jpg', bbox_inches = 'tight')
    plt.show()
    plt.close()

    plt.plot(epochs, train_f1scores, color = 'blue')
    plt.plot(epochs, val_f1scores, color = 'red')
    plt.axhline(y=scores['Training F1Score'], linestyle='dotted', color = 'blue')
    plt.axhline(y=scores['Validation F1Score'], linestyle='dotted', color = 'red')
    plt.gca().legend(('Training F1Score: {}'.format(scores['Training F1Score']),
                      'Validation F1Score: {}'.format(scores['Validation F1Score'])))
    plt.title("Training-Validation F1Score Curve - Sentence Extraction")
    plt.xlabel("Epoch")
    plt.ylabel("F1Score")
    plt.savefig('epoch_trainval_f1scores_se.jpg', bbox_inches = 'tight')
    plt.show()
    plt.close()

    val_outputs_df = pd.DataFrame.from_dict(val_outputs)


    return scores, val_outputs_df

def print_stats_se(sentence_model_name, features_dim, pooling, scores):
    print("Sentence Extraction - Statistics")
    print(scores)
    print("--------------------------------------------------------------------")

    stats = {}
    stats['Sentence Model'] = sentence_model_name
    stats['FEATURES_DIM']  = features_dim
    stats['HIDDEN_DIM'] = SE_HIDDEN_DIM
    stats['POOLING'] = pooling
    stats['Optimizer'] = SE_OPTIMIZER_NAME
    stats['Loss'] = SE_LOSS_FUNCTION_NAME
    stats['Learning Rate'] = SE_LEARNING_RATE
    stats['Weight Decay'] = SE_WEIGHT_DECAY
    stats['DROPOUT'] = SE_DROPOUT
    stats['THRESHOLD'] = SE_THRESHOLD
    stats['epoch'] = SE_EPOCH_MAX
    stats['SE_BALANCE_NEG_MULTIPLIER'] = SE_BALANCE_NEG_MULTIPLIER
    stats['SE_BERT_NUM_ATTENTION_HEADS'] = SE_BERT_NUM_ATTENTION_HEADS
    stats['SE_DROPOUT_HIDDEN'] = SE_DROPOUT_HIDDEN
    stats['SE_DROPOUT_ATTENTION'] = SE_DROPOUT_ATTENTION

    stats.update(scores)

    # write model parameters to file
    with open('score_se.csv', 'w') as csvfile:  
        writer = csv.writer(csvfile)
        for key, value in stats.items():
           writer.writerow([key, value])

    # Append model parameters to file for iterative experiments
    with open('scores_se.csv', 'a', encoding='utf8', newline='') as csvfile:
        fc = csv.DictWriter(csvfile, fieldnames=stats.keys(),)

        if csvfile.tell() == 0:
            fc.writeheader()

        fc.writerow(stats)

    return stats



def process_se(train_df, val_df):
  
  scores_se = []
  val_outputs_df = None
  pbar = tqdm_notebook(range(len(SE_MODELS)), desc="SE Training using Huggingface Model")
  for i in pbar:
    scores = {}
    
    sentence_model_name = SE_MODELS[i]["name"]
    features_dim = SE_MODELS[i]["features_dim"]
    pooling = SE_MODELS[i]["pooling"]

    pbar.set_postfix_str(sentence_model_name)

    try:
      model, loss_function, optimizer, tokenizer = create_model_se(sentence_model_name, features_dim, pooling)
      train_loader, classWeightsTensor = get_data_se(tokenizer, train_df, True if SE_BALANCE_NEG_MULTIPLIER > 0 else False, True)
      val_loader, _ = get_data_se(tokenizer, val_df, False, False)
      
      scores, val_outputs_df = train_se(model, loss_function, optimizer, tokenizer, train_loader, val_loader, val_df, classWeightsTensor)

      val_outputs_df.to_csv("se_bcs_pred_output.csv")
      
    except Exception as ex:
      scores["Error"] = '{}'.format(ex)
      print(scores["Error"])

    print_stats_se(sentence_model_name, features_dim, pooling, scores)

  return scores_se, val_outputs_df
    

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.3 MB/s 
     |████████████████████████████████| 182 kB 90.3 MB/s 
     |████████████████████████████████| 7.6 MB 77.8 MB/s 


In [8]:
'''
  Question&Answering Dataset and Dataloader cell
'''
from torch.utils.data import Dataset, DataLoader 


# Dataset for ClickBaitsWebis22 (CBW22) Question Answering (QA)
class CBW22QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        batch = {}
        for key, val in self.encodings.items():
            if type(val[idx]) == str or isinstance(val[idx],tuple):
                batch[key] = val[idx]
            else:
                batch[key] = torch.tensor(val[idx]).to(device)
        return batch

    def __len__(self):
        return len(self.encodings.input_ids)

# get context and question encodings from the CBW22 data
def get_encodings(tokenizer, df, document_context):
    contexts = []
    if document_context == True:
      contexts = df['tgpDocument'].tolist()
    else:
      contexts = df['targetParagraphs'].tolist()
    
    questions = df['postText'].tolist()
    tgpIds = df['tgpId'].tolist()
    encodingIds = np.arange(len(df))
    
    encodings = tokenizer(contexts, questions, truncation=True, padding=True)
    encodings.update({'encodingIds': encodingIds, 'tgpIds': tgpIds, "contexts":contexts, "questions": questions})
    return encodings

def add_answer_positions_to_encodings(tokenizer, encodings, df):
    answers = df['tgpQALabel'].tolist()
    
    # initialize lists to contain the token indices of answer start/end
    start_token_positions = []
    end_token_positions = []
    start_char_positions = []
    end_char_positions = []
    
    no_spoilers_count = 0
    truncated_count = 0
    end_pos_shifted = 0
    for i in range(len(answers)):
        if len(answers[i]['spoiler']) > 0:
            # append start/end token position using char_to_token method
            start_token_positions.append(encodings.char_to_token(i, answers[i]['start_pos']))
            end_token_positions.append(encodings.char_to_token(i, answers[i]['end_pos']-1))
            
            # if start position is None, the answer passage has been truncated
            if start_token_positions[-1] is None:
                truncated_count += 1
                start_token_positions[-1] = tokenizer.model_max_length
            
            # end position cannot be found, char_to_token found space, so shift position until found
            if end_token_positions[-1] is None:
              end_pos_shifted +=1
              shift = 1
              while end_token_positions[-1] is None:
                  end_token_positions[-1] = encodings.char_to_token(i, answers[i]['end_pos'] - shift)
                  shift += 1
                  
            start_char_positions.append(answers[i]['start_pos'])
            end_char_positions.append(answers[i]['end_pos'])
        else:
            start_char_positions.append(answers[i]['start_pos'])
            end_char_positions.append(answers[i]['end_pos'])
            start_token_positions.append(0)
            end_token_positions.append(0)
            
            no_spoilers_count += 1
        
    #print("Number of No Spoilers found (TopN Rouge excluded): {}".format(no_spoilers_count)) 
    #print("Number of truncated documents by tokenizer: {}".format(truncated_count)) 
    #print("Number of endshifted spoilers by tokenizer: {}".format(end_pos_shifted)) 

    # update our encodings object with the new token-based start/end positions
    encodings.update({
        'start_token_positions': start_token_positions, 
        'end_token_positions': end_token_positions,
        'start_char_positions':start_char_positions, 
        'end_char_positions':end_char_positions
    })

def mark_rouge_topn_tgp_for_sentence_qa(record):
    topn_tgp_marked = [False] * len(record['targetParagraphs'])
    
    if TOP_N_ROUGE_SCORES > 0:
        scores = record['tgpRougeScore']
        topn_max  = len(scores) if len(scores) < TOP_N_ROUGE_SCORES else TOP_N_ROUGE_SCORES
        topn_scores = scores[0:topn_max]
        topn_indexes = [score['index'] for score in topn_scores]
        topn_indexes.sort()
        
        for topn_index in topn_indexes:
            topn_tgp_marked[topn_index] = True
    else:
        topn_tgp_marked = [True] * len(record['targetParagraphs'])
    
    return topn_tgp_marked

"""
  the original dataset has spoilerPositions which define the answer location i.e targetParagraphIndex and character span in that targetParagraph
  There is another field 'spoiler' which has extracted text as answer from the targetParagraph
  However, i have noticed that there is sometimes (1087 times to be precise) discripancy between the spoiler text provided and text extracted from 
  targetParagraph by using the spoilerPosition information. This text spoiler is required to calculate the BLEU score.
  Hence, i am disregarding the provided spoiler text and reconstructing it using the spoilerPositions 
"""
def compute_qa_answer_label_statement(record):
    answerLabels = [{}] * record['tgpsCount']
    
    for i in range(len(record['spoiler'])):
        spoiler = record['spoiler'][i]
        spoilerPosition = record['spoilerPositions'][i]
        tgpSpoilerStartIndex = spoilerPosition[0][0] # 0th index defines targetParagraphs index
        tgpSpoilerEndIndex = tgpSpoilerStartIndex
        
        if len(spoilerPosition) == 2:
            tgpSpoilerEndIndex = spoilerPosition[1][0] # 0th     index defines targetParagraphs index
       
        if tgpSpoilerEndIndex == tgpSpoilerStartIndex:
            if spoiler in record['targetParagraphs'][tgpSpoilerStartIndex]:
                start_pos = record['targetParagraphs'][tgpSpoilerStartIndex].index(spoiler)
                end_pos = start_pos + len(spoiler)
                
                answerLabels[tgpSpoilerStartIndex] = {
                    "spoiler": record['targetParagraphs'][tgpSpoilerStartIndex][start_pos:end_pos],
                    "start_pos": start_pos, 
                    "end_pos": end_pos
                }
        else:
            
            spoilerTokens = spoiler.split()
            for i in range(len(spoilerTokens), -1, -1):
                searchSpoiler = " ".join(spoilerTokens[0:i])
                if searchSpoiler in record['targetParagraphs'][tgpSpoilerStartIndex]:
                    start_pos = record['targetParagraphs'][tgpSpoilerStartIndex].index(searchSpoiler)
                    end_pos = len(record['targetParagraphs'][tgpSpoilerStartIndex])
                    
                    answerLabels[tgpSpoilerStartIndex] = {
                        "spoiler": record['targetParagraphs'][tgpSpoilerStartIndex][start_pos:end_pos],
                        "start_pos": start_pos, 
                        "end_pos": end_pos
                    }   
                    break
                
                
            for i in range(0, len(spoilerTokens)):
                searchSpoiler = " ". join(spoilerTokens[i:len(spoilerTokens)])
                if searchSpoiler in record['targetParagraphs'][tgpSpoilerEndIndex]:
                    start_pos = 0
                    end_pos = len(searchSpoiler)
                    
                    answerLabels[tgpSpoilerEndIndex] = {
                        "spoiler": record['targetParagraphs'][tgpSpoilerEndIndex][start_pos:end_pos],
                        "start_pos": start_pos, 
                        "end_pos": end_pos
                    }   
                    break
       
    return answerLabels

def compute_qa_answer_label_document(record):
    # phrase and passage has only 1 spoiler
    spoilerPosition = record['spoilerPositions'][0]
    
    spoiler = record['spoiler'][0]
    
    start_pos = 0
    end_pos = 0
    tgpDocument = record['tgpDocument']
    if spoiler in tgpDocument:
        start_pos = tgpDocument.index(spoiler)
        end_pos = start_pos + len(spoiler)
    
    answer = {
        "spoiler": record['tgpDocument'][start_pos:end_pos],
        "start_pos": start_pos, 
        "end_pos": end_pos
    }   
    return answer

def mark_spoiler_exists_flag(answerLabel):
    return "spoiler" in answerLabel.keys()

def prepare_data_qa(tokenizer, df, batch_size, document_context):
  col_list = ['tgpId', 'postText', 'targetParagraphs' ,'tgpDocument', 'tgpQALabel']
  df = df[col_list]

  encodings = get_encodings(tokenizer, df, document_context)

  #append spoiler positions 
  add_answer_positions_to_encodings(tokenizer, encodings, df)

  ds = CBW22QADataset(encodings)
  dl = DataLoader(batch_size=batch_size, shuffle=True, dataset=ds)
  return encodings, dl

def reset_tgp_id(record):
    return (record['uuid'],-1)

def prepare_data_qa_document(tokenizer, df, batch_size):
  qad_df = df.copy()
  qad_df['tgpQALabel'] = qad_df.apply(compute_qa_answer_label_document, axis = 1)
  qad_df['tgpId'] = qad_df.apply(reset_tgp_id, axis = 1)
  return prepare_data_qa(tokenizer, qad_df, batch_size, True)
  
def get_dataloader_qa_statement(tokenizer, df, batch_size):
    df = df[df.tgpSELabel == 1]
    # get only those targetParagraphs which appear in spoiler
    df['spoilerExists'] = df['tgpQALabel'].apply(mark_spoiler_exists_flag)
    df = df[df.spoilerExists == True]
    return prepare_data_qa(tokenizer, df, batch_size, False)
  
def prepare_data_qa_statement(tokenizer, df, batch_size):
  df['tgpQALabel'] = df.apply(compute_qa_answer_label_statement, axis = 1)
  col_list = ['tgpId','postText', 'targetParagraphs', 'tgpDocument', 'tgpSELabel', 'tgpQALabel']
  df = df[col_list]
  df.reset_index(level=0, inplace=True)
  df = df.apply(pd.Series.explode)
  return get_dataloader_qa_statement(tokenizer, df, batch_size)

In [9]:
'''
    Question Answering Block for  :
        1. factory method of model creation
        2. train loop - train_epoch and validate_epoch
        3. prediction
'''
from string import punctuation

try:
  from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoConfig
except ModuleNotFoundError:
  !pip install transformers
  from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoConfig

try:
  import sentencepiece
except ModuleNotFoundError:
  !pip install sentencepiece
  import sentencepiece

try:
  from nltk.translate import bleu
except ModuleNotFoundError:
  !pip install nltk
  from nltk.translate import bleu

import math
from tqdm.notebook import tqdm_notebook
import numpy as np
from string import punctuation
import os
import torch
import matplotlib.pyplot as plt
import torch.optim as optim 
from transformers import AdamW
from transformers import get_scheduler


def create_model_qa(model_name, dl_len):
    configuration = AutoConfig.from_pretrained(model_name)
    configuration.hidden_dropout_prob = QA_DROPOUT_HIDDEN
    configuration.attention_probs_dropout_prob = QA_DROPOUT_ATTENTION
    configuration.num_hidden_layers = QA_BERT_NUM_HIDDEN_LAYERS
    configuration.num_attention_heads = QA_BERT_NUM_ATTENTION_HEADS
    model = AutoModelForQuestionAnswering.from_pretrained(pretrained_model_name_or_path = model_name, config = configuration)
    
    model.to(device)

    if QA_OPTIMIZER_NAME == "SGD":
      optimizer = optim.SGD(model.parameters(), lr = QA_LEARNING_RATE, weight_decay = QA_WEIGHT_DECAY)
    elif QA_OPTIMIZER_NAME == "Adam":
      optimizer = optim.Adam(model.parameters(), lr = QA_LEARNING_RATE, weight_decay = QA_WEIGHT_DECAY)
    elif QA_OPTIMIZER_NAME == "AdamW":
      optimizer = optim.AdamW(model.parameters(), lr = QA_LEARNING_RATE, weight_decay = QA_WEIGHT_DECAY)
      #optimizer = AdamW(model.parameters(), lr=QA_LEARNING_RATE)

    lr_scheduler = None
    if QA_LEARNING_RATE_SCHEDULER == True:
      num_training_steps = QA_EPOCH_MAX * dl_len
      lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
    
    return model, optimizer, lr_scheduler

def compute_scores_qa(outputs, batch, encodings):
    encodingIds = batch['encodingIds'].detach().cpu().numpy()
    contexts = batch['contexts']
    
    pred_outputs = {
        "tgpDocument": contexts,
        "postText": batch['questions'],
        "spoiler": [],
        "predictions": [],
        "bleuscore" : [],
        "spoiler_punct_strip": [],
        "predictions_punct_stip": [],
        "bluescore_punct_strip":[],
        "token span accuracy": []
    }

    start_positions = batch['start_token_positions'].to(device)
    end_positions = batch['end_token_positions'].to(device) 
    start_char_positions = batch['start_char_positions'].detach().cpu().numpy()
    end_char_positions = batch['end_char_positions'].detach().cpu().numpy()

    # pull prediction tensors out and argmax to get predicted tokens
    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)

    start_pred  = start_pred.detach().cpu().numpy()
    end_pred  = end_pred.detach().cpu().numpy()

    for i in range(len(encodingIds)):
      encodingIndex = encodingIds[i]
      text = contexts[i]

      # calculate accuracy for both and append to accuracy list
      accuracy_start_token = 100 if (start_pred[i] == start_positions[i]) else 0 
      accuracy_end_token = 100 if (end_pred[i] == end_positions[i]) else 0 
      batch_accuracy_token = (accuracy_start_token + accuracy_end_token)/2
      pred_outputs["token span accuracy"].append(batch_accuracy_token)

      pred_start_char_pos = 0
      pred_end_char_pos = 0
      
      pos = encodings.token_to_chars(encodingIndex, start_pred[i])
      if pos != None:
          pred_start_char_pos = pos.start
         
      pos = encodings.token_to_chars(encodingIndex, end_pred[i])
      if pos != None:
          pred_end_char_pos = pos.end
      
      predict_answer = text[pred_start_char_pos:pred_end_char_pos].strip()
      pred_outputs["predictions"].append(predict_answer)
      target_answer = text[start_char_positions[i]:end_char_positions[i]].strip()
      pred_outputs["spoiler"].append(target_answer)

      bleu_score = bleu([predict_answer.split()], target_answer.split(), (1,),) * 100
      pred_outputs["bleuscore"].append(bleu_score)

      predict_answer_punctstrip = predict_answer.strip(punctuation)
      pred_outputs["predictions_punct_stip"].append(predict_answer_punctstrip)
      targets_punctstrip = target_answer.strip(punctuation)
      pred_outputs["spoiler_punct_strip"].append(targets_punctstrip)

      bleu_score_punctstrip = bleu([predict_answer_punctstrip.split()], targets_punctstrip.split(), (1,),) * 100
      pred_outputs["bluescore_punct_strip"].append(bleu_score_punctstrip)
    
   
    batch_accuracy_token = sum(pred_outputs["token span accuracy"])/len(pred_outputs["token span accuracy"])
    batch_bleuscore = sum(pred_outputs["bleuscore"])/len(pred_outputs["bleuscore"])
    batch_bleuscore_punctstrip = sum(pred_outputs["bluescore_punct_strip"])/len(pred_outputs["bluescore_punct_strip"])

    return batch_accuracy_token, batch_bleuscore, batch_bleuscore_punctstrip, pred_outputs

def train_epoch_qa(epoch, model, optimizer, lr_scheduler, train_loader, train_encodings):
    epoch_loss = []
    epoch_accuracies_token = []
    epoch_bleu_scores = []
    epoch_bleu_scores_punctstrip = []
    
    model.train()
    
    pbar = tqdm_notebook(train_loader, desc="Training batch for epoch-{}".format(epoch))
    for _,batch in enumerate(pbar):
        # initialize calculated gradients (from prev step)
        optimizer.zero_grad()
        
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_token_positions'].to(device)
        end_positions = batch['end_token_positions'].to(device)
        
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optimizer.step()

        if lr_scheduler != None:
          lr_scheduler.step()
        
        batch_loss = loss.item()
        if math.isnan(batch_loss) == False:
          epoch_loss.append(batch_loss)
        
        batch_accuracy_token, batch_bleuscore, batch_bleuscore_punct, _ = compute_scores_qa(outputs, batch, train_encodings)
        epoch_accuracies_token.append(batch_accuracy_token)
        epoch_bleu_scores.append(batch_bleuscore)
        epoch_bleu_scores_punctstrip.append(batch_bleuscore_punct)
        pbar.set_postfix_str("Loss: {}, BleuScore:{}".format(round(batch_loss,2), round(batch_bleuscore,2)))
    
    loss = 0
    if len(epoch_loss) > 0 :
      loss = sum(epoch_loss) / len(epoch_loss)
    
    accuracy_token = 0
    if len(epoch_accuracies_token) > 0:
      accuracy_token = sum(epoch_accuracies_token)/len(epoch_accuracies_token)

    bleu_score = 0
    if len(epoch_bleu_scores) > 0:
      bleu_score = sum(epoch_bleu_scores)/len(epoch_bleu_scores)
    
    bleu_score_punctstrip = 0
    if len(epoch_bleu_scores_punctstrip) > 0:
      bleu_score_punctstrip = sum(epoch_bleu_scores_punctstrip)/len(epoch_bleu_scores_punctstrip)

    #print("Loss: {}, TokenAccuracy:{}, BleuScore:{}, BleuScorePunctRemove:{}".format(round(loss,2), round(accuracy_token,2), round(bleu_score,2),round(bleu_score_punctstrip,2) ))
    
    return loss, accuracy_token, bleu_score, bleu_score_punctstrip


def predict_epoch_qa(epoch, model, optimizer, val_loader, val_encodings, output_filename):
    epoch_loss = []
    epoch_accuracies_token = []
    epoch_bleu_scores = []
    epoch_bleu_scores_punctstrip = []
    epoch_outputs = []
    
    model.eval()
    
    pbar = tqdm_notebook(val_loader, desc="Validation batch for epoch-{}".format(epoch))
    for _,batch in enumerate(pbar):
        
        # we don't need to calculate gradients as we're not training
        with torch.no_grad():
        
            # pull all the tensor batches required for training
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_token_positions'].to(device)
            end_positions = batch['end_token_positions'].to(device)
            
            
            # train model on batch and return outputs (incl. loss)
            outputs = model(input_ids, attention_mask=attention_mask,
                            start_positions=start_positions,
                            end_positions=end_positions)
            
            # extract loss
            loss = outputs[0]

            batch_loss = loss.item()
            if math.isnan(batch_loss) == False:
              epoch_loss.append(batch_loss)
            
            batch_accuracy_token, batch_bleuscore, batch_bleuscore_punct, batch_outputs = compute_scores_qa(outputs, batch, val_encodings)

            epoch_outputs.append(batch_outputs)
            epoch_accuracies_token.append(batch_accuracy_token)
            epoch_bleu_scores.append(batch_bleuscore)
            epoch_bleu_scores_punctstrip.append(batch_bleuscore_punct)

            pbar.set_postfix_str("Loss: {}, BleuScore:{}".format(round(batch_loss,2), round(batch_bleuscore,2)))
   
    loss = 0
    if len(epoch_loss) > 0 :
      loss = sum(epoch_loss) / len(epoch_loss)
    
    accuracy_token = 0
    if len(epoch_accuracies_token) > 0:
      accuracy_token = sum(epoch_accuracies_token)/len(epoch_accuracies_token)

    bleu_score = 0
    if len(epoch_bleu_scores) > 0:
      bleu_score = sum(epoch_bleu_scores)/len(epoch_bleu_scores)
    
    bleu_score_punctstrip = 0
    if len(epoch_bleu_scores_punctstrip) > 0:
      bleu_score_punctstrip = sum(epoch_bleu_scores_punctstrip)/len(epoch_bleu_scores_punctstrip)

    #print("Loss: {}, TokenAccuracy:{}, BleuScore:{}, BleuScorePunctRemove:{}".format(round(loss,2), round(accuracy_token,2), round(bleu_score,2),round(bleu_score_punctstrip,2) ))

    df = pd.DataFrame.from_dict(epoch_outputs)
    df = df.apply(pd.Series.explode)
    df.to_csv(output_filename)

    return loss, accuracy_token, bleu_score, bleu_score_punctstrip


def train_qa(model, optimizer, lr_scheduler, train_loader, val_loader, train_encodings, val_encodings, output_filename):
    
    train_losses = []
    train_accuracies_token = []
    train_bleuscores = []
    train_bleuscores_punctstrip = []

    val_losses = []
    val_accuracies_token = []
    val_bleuscores = []
    val_bleuscores_punctstrip = []

    epochs = np.arange(QA_EPOCH_MAX)

    pbar = tqdm_notebook(range(QA_EPOCH_MAX), desc="Epoch", position = 0)
    for epoch in pbar:
      
      epoch_loss_train, epoch_accuracy_token, epoch_bleu_score_train, epoch_bleu_score_punctstrip = train_epoch_qa(epoch, model, optimizer, lr_scheduler, train_loader, train_encodings)
      train_losses.append(epoch_loss_train)
      train_accuracies_token.append(epoch_accuracy_token)
      train_bleuscores.append(epoch_bleu_score_train)
      train_bleuscores_punctstrip.append(epoch_bleu_score_punctstrip)

      epoch_loss_val, epoch_accuracy_token, epoch_bleu_score_val, epoch_bleu_score_punctstrip = predict_epoch_qa(epoch, model, optimizer, val_loader, val_encodings, output_filename)
      val_losses.append(epoch_loss_val)
      val_accuracies_token.append(epoch_accuracy_token)
      val_bleuscores.append(epoch_bleu_score_val)
      val_bleuscores_punctstrip.append(epoch_bleu_score_punctstrip)
     
      pbar.set_postfix_str("Training loss: {} / BleuScore: {}, Validation Loss: {} / BleuScore: {}".format(
              round(epoch_loss_train,2), round(epoch_bleu_score_train,2), 
              round(epoch_loss_val,2),  round(epoch_bleu_score_val,2)))

    scores = {}
    scores['Training Loss'] = round(train_losses[-1], 2)
    scores['Training Accuracy Token'] = round(train_accuracies_token[-1], 2)
    scores['Training BleuScore'] = round(train_bleuscores[-1], 2)
    scores['Training BleuScore Punctuation Strip'] = round(train_bleuscores_punctstrip[-1], 2)

    scores['Validation Loss'] = round(val_losses[-1], 2)
    scores['Validation Accuracy Token'] = round(val_accuracies_token[-1], 2)
    scores['Validation BleuScore'] = round(val_bleuscores[-1], 2)
    scores['Validation BleuScore Punctuation Strip'] = round(val_bleuscores_punctstrip[-1], 2)

    plt.plot(epochs, train_losses, color = 'blue')
    plt.plot(epochs, val_losses, color = 'red')
    plt.axhline(y=scores['Training Loss'], linestyle='dotted', color = 'blue')
    plt.axhline(y=scores['Validation Loss'], linestyle='dotted', color = 'red')
    plt.gca().legend(('Training Loss: {}'.format(scores['Training Loss']),
                      'Validation Loss: {}'.format(scores['Validation Loss'])))
    plt.title("Training-Validation Loss Curve - Question & Answering")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.savefig('epoch_trainval_losses_qa.jpg', bbox_inches = 'tight')
    plt.show()
    plt.close()

    plt.plot(epochs, train_accuracies_token, color = 'blue')
    plt.plot(epochs, val_accuracies_token, color = 'red')
    plt.axhline(y=scores['Training Accuracy Token'], linestyle='dotted', color = 'blue')
    plt.axhline(y=scores['Validation Accuracy Token'], linestyle='dotted', color = 'red')
    plt.gca().legend(('Training Accuracy Token-Span: {}'.format(scores['Training Accuracy Token']),
                      'Validation Accuracy Token-Span: {}'.format(scores['Validation Accuracy Token'])))
    plt.title("Training-Validation Accuracy Token-Span Curve - Question & Answering")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.savefig('epoch_trainval_accuracy_tokenspan_qa.jpg', bbox_inches = 'tight')
    plt.show()
    plt.close()

    """
    plt.plot(epochs, train_bleuscores, color = 'blue')
    plt.plot(epochs, val_bleuscores, color = 'red')
    plt.axhline(y=scores['Training BleuScore'], linestyle='dotted', color = 'blue')
    plt.axhline(y=scores['Validation BleuScore'], linestyle='dotted', color = 'red')
    plt.gca().legend(('Training BleuScore: {}'.format(scores['Training BleuScore']),
                      'Validation BleuScore: {}'.format(scores['Validation BleuScore'])))
    plt.title("Training-Validation BleuScore Curve - Question & Answering")
    plt.xlabel("Epoch")
    plt.ylabel("BleuScore")
    #plt.show()
    plt.savefig('epoch_trainval_accuracy_bleuscore_qa.jpg', bbox_inches = 'tight')
    plt.close()
    """
    
    plt.plot(epochs, train_bleuscores_punctstrip, color = 'blue')
    plt.plot(epochs, val_bleuscores_punctstrip, color = 'red')
    plt.axhline(y=scores['Training BleuScore Punctuation Strip'], linestyle='dotted', color = 'blue')
    plt.axhline(y=scores['Validation BleuScore Punctuation Strip'], linestyle='dotted', color = 'red')
    plt.gca().legend(('Training BleuScore: {}'.format(scores['Training BleuScore Punctuation Strip']),
                      'Validation BleuScore: {}'.format(scores['Validation BleuScore Punctuation Strip'])))
    plt.title("Training-Validation BleuScore Curve - Question & Answering")
    plt.xlabel("Epoch")
    plt.ylabel("BleuScore")
    plt.savefig('epoch_trainval_accuracy_bleuscore_punctstrip_qa.jpg', bbox_inches = 'tight')
    plt.show()
    plt.close()
    
    return scores

import csv
def print_stats(model_name, train_scores):
    stats = {}
    stats['Model Name'] = model_name
    stats['Optimizer'] = QA_OPTIMIZER_NAME
    stats['QA_DROPOUT_HIDDEN'] = QA_DROPOUT_HIDDEN
    stats['QA_DROPOUT_ATTENTION'] = QA_DROPOUT_ATTENTION
    stats['Learning Rate'] = QA_LEARNING_RATE
    stats['Weight Decay'] = QA_WEIGHT_DECAY
    stats['TOP_N_ROUGE_SCORES'] = TOP_N_ROUGE_SCORES
    stats['REMOVE_PUNCTUATIONS'] = REMOVE_PUNCTUATIONS
    stats['epoch'] = QA_EPOCH_MAX
    
    stats.update(train_scores)

    # write model parameters to file
    with open('score_qa.csv', 'w') as csvfile:  
        writer = csv.writer(csvfile)
        for key, value in stats.items():
           writer.writerow([key, value])

    # Append model parameters to file for iterative experiments
    with open('scores_qa.csv', 'a', encoding='utf8', newline='') as csvfile:
        fc = csv.DictWriter(csvfile, fieldnames=stats.keys(),)

        if csvfile.tell() == 0:
            fc.writeheader()

        fc.writerow(stats)
  
'''
    Block for Main Loop
'''

def process_qa_document(train_df, val_df):
  scores_qa = []

  pbar = tqdm_notebook(range(len(QA_MODELS)), desc="QA-Document Training using Huggingface Model")
  for i in pbar:
    scores = {}
    
    qa_model_name = QA_MODELS[i]["name"]
    pbar.set_postfix_str(qa_model_name)

    try:
      tokenizer = AutoTokenizer.from_pretrained(qa_model_name)

      train_qad_encodings, train_qad_loader = prepare_data_qa_document(tokenizer, train_df, QA_TRAIN_BATCH_SIZE)
      val_qad_encodings, val_qad_loader = prepare_data_qa_document(tokenizer, val_df, QA_VAL_BATCH_SIZE)

      # create mode
      model, optimizer, lr_scheduler = create_model_qa(qa_model_name, len(train_qad_loader))

      #train
      scores = train_qa(model, optimizer, lr_scheduler, train_qad_loader, val_qad_loader, train_qad_encodings, val_qad_encodings, 'output_qa_document.csv')

      print("Question Answering Document - Statistics")
      print(scores)
      print_stats(qa_model_name, scores)
      print("--------------------------------------------------------------------")

      model_path = 'models/{}-doc-cbw22-djoshi'.format(qa_model_name)
      model.save_pretrained(model_path)
      tokenizer.save_pretrained(model_path)
    except Exception as ex:
      scores["Error"] = '{}'.format(ex)
      print(scores["Error"])

    scores_qa.append(scores)

    return scores

def process_qa_statement(train_df, val_df, se_val_outputs_df):
  scores_qa = []

  pbar = tqdm_notebook(range(len(QA_MODELS)), desc="QA-Statement Training using Huggingface Model")
  for i in pbar:
    scores = {}
    
    qa_model_name = QA_MODELS[i]["name"]
    pbar.set_postfix_str(qa_model_name)

    try:
      tokenizer = AutoTokenizer.from_pretrained(qa_model_name)

      train_qad_encodings, train_qad_loader = prepare_data_qa_statement(tokenizer, train_df, QA_TRAIN_BATCH_SIZE)
      val_qad_encodings, val_qad_loader = prepare_data_qa_statement(tokenizer, val_df, QA_VAL_BATCH_SIZE)

      # create mode
      model, optimizer, lr_scheduler = create_model_qa(qa_model_name, len(train_qad_loader))

      #train
      scores = train_qa(model, optimizer, lr_scheduler, train_qad_loader, val_qad_loader, train_qad_encodings, val_qad_encodings, 'output_qa_statement.csv')

      print("Question Answering Statement - Statistics")
      print(scores)
      print_stats(qa_model_name, scores)
      print("--------------------------------------------------------------------")

      model_path = 'models/{}-statement-cbw22-djoshi'.format(qa_model_name)
      model.save_pretrained(model_path)
      tokenizer.save_pretrained(model_path)

      #predict SE output
      if se_val_outputs_df is not None:
        print("Predicting Spoiler Spans for spoiler statements predicted by BertSentenceClass Extractor...")
        se_val_outputs_encodings, se_outputs_loader = get_dataloader_qa_statement(tokenizer, se_val_outputs_df, QA_VAL_BATCH_SIZE)
        loss, accuracy_token, bleu_score, bleu_score_punctstrip = predict_epoch_qa(-1, model, optimizer, se_outputs_loader, se_val_outputs_encodings, 'output_se_output_qa_statement.csv')
        print("Loss: {}, TokenAccuracy:{}, BleuScore:{}, BleuScorePunctRemove:{}".format(round(loss,2), round(accuracy_token,2), round(bleu_score,2),round(bleu_score_punctstrip,2) ))
    except Exception as ex:
      scores["Error"] = '{}'.format(ex)
      print(scores["Error"])

    scores_qa.append(scores)

    return scores

  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 8.4 MB/s 


In [10]:
'''
    Block for Main Loop
'''
import os
import torch
import random
import numpy as np

from transformers import logging
logging.set_verbosity_error()

def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def load_device():
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    device = torch.device(device)
    print("Device = {}".format(device))    
    return device



seed_torch()
device = load_device()

# load json files in to pandas dataframe and curate
print("Loading Training and Validation Data...")
train_df, train_phrase_passage_df, train_multi_df = load_data('train.jsonl')
val_df, val_phrase_passage_df, val_multi_df = load_data('validation.jsonl')

se_val_outputs_df = None
if PERFORM_DOCUMENT_QA == True:
  set_qa_document_configuration()
  process_qa_document(train_phrase_passage_df, val_phrase_passage_df)

if PERFORM_SENTENCE_EXTRACTION == True:
  set_se_configuration()
  scores_se, se_val_outputs_df = process_se(train_df, val_df)
  if se_val_outputs_df is not None:
    append_info_to_se_output(val_df, se_val_outputs_df)
    se_val_outputs_df.to_csv("se_val_outputs_df.csv")

if PERFORM_SENTENCE_QA == True:
  set_qa_statement_configuration()
  process_qa_statement(train_phrase_passage_df, val_phrase_passage_df, se_val_outputs_df)

  

print("Finished Processing")

Device = cuda
Loading Training and Validation Data...


QA-Document Training using Huggingface Model:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

KeyboardInterrupt: ignored